In [38]:
import json
import pymysql

with open('..\security\parameters.json', 'r') as json_file:
    parameters = json.load(json_file)

print(parameters)

{'bucket_name': 'ebarahonabucket01', 'DB_USER': 'myguest', 'DB_PASS': 'jA$9<,C]TN`1JCxV', 'DB_NAME': 'mydbpoc', 'DB_HOST': '34.121.213.191', 'DB_PORT': '3306'}


In [39]:
db_user = parameters['DB_USER']
db_pass = parameters['DB_PASS']
db_name = parameters['DB_NAME']
db_host = parameters['DB_HOST']
db_port = parameters['DB_PORT']

def connect_database (db_user, db_pass, db_name, db_host):
    connection = pymysql.connect(
        host        = db_host,
        user        = db_user, 
        password    = db_pass,
        db          = db_name,
    )
    cursor = connection.cursor()
    return connection, cursor

def str_columns(columns_query):
    lc = len(columns_query)
    strc = ''
    val = ''
    for index, column in enumerate(columns_query):
        if index == 0:
            strc = strc + '(' + column  + ', '  
            val = val + '(%s, ' 
        elif index < lc - 1:
            strc = strc + column + ', ' 
            val = val + '%s, '
        elif index == lc - 1:
            strc = strc + column + ')'
            val = val + '%s)'
    return strc, val

def check_table(connection, table_name):
    cursor = connection.cursor()
    cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(table_name.replace('\'', '\'\'')))
    if cursor.fetchone()[0] == 1:
        cursor.close()
        return True
    cursor.close()
    return False

def clean_create_table (connection, table_name, columns_table, columns_types):
    cursor = connection.cursor()
    if check_table(connection, table_name):
        query = ("TRUNCATE TABLE {0}" ).format(table_name)    
    else:
        #Cambia el tipo de dato para lo 'str'  
        ilist = columns_types.index('str')
        columns_types = columns_types[:ilist]+['varchar(100)']+columns_types[ilist+1:]
        lc = len(columns_table)
        strc = ''
        val = ''
        for index, column in enumerate(columns_table):
            if index == 0:
                strc = strc + '(' + column  + ' ' + columns_types[index] + ', '
            elif index < lc - 1:
                strc = strc + column + ' ' + columns_types[index] + ', '
            elif index == lc - 1:
                strc = strc + column + ' ' + columns_types[index] + ')'
        query = ("CREATE TABLE {0} {1}" ).format(table_name, strc)
    cursor.execute(query)
    connection.commit()
    cursor.close()  

def query_batch(strc, val):
    query = ("INSERT INTO mytable {0} VALUES {1}").format(strc, val)
    return query

def load_orders(connection, query, data, batch_size=10000):
    cursor = connection.cursor()
    for i in range(0, len(data), batch_size):
        cursor.executemany(query, data[i:i+batch_size])
        connection.commit()
        print('Datos cargados...')
    cursor.close()

In [40]:
conn, curs = connect_database (db_user, db_pass, db_name, db_host)

In [41]:
import pandas as pd

name = ['Juan', 'Laura', 'Pepe']
id = [42, 40, 37]

data_mytable = pd.DataFrame([], columns=['id', 'name'])
data_mytable['id'] = id
data_mytable['name'] = name

print(data_mytable)

   id   name
0  42   Juan
1  40  Laura
2  37   Pepe


In [42]:
list(range(0, len(data_mytable.values.tolist()), 10000))

[0]

In [43]:
columns_query = ['id', 'name']
lc = len(columns_query)
print(lc)
strc = ''
val = ''
for index, column in enumerate(columns_query):
    if index == 0:
        strc = strc + '(' + column  + ', '  
        val = val + '(%s, ' 
    elif index < lc - 1:
        strc = strc + column + ', ' 
        val = val + '%s, '
    elif index == lc - 1:
        strc = strc + column + ')'
        val = val + '%s)'

query = ("INSERT INTO mytable {0} VALUES {1}").format(strc, val)

print(query)

2
INSERT INTO mytable (id, name) VALUES (%s, %s)
